In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Define function to bin rare categories
def bin_rare_categories(df, column, threshold):
    value_counts = df[column].value_counts()
    rare_categories = value_counts[value_counts < threshold].index
    df[column] = df[column].replace(rare_categories, 'Other')
    return df

# Display the first few rows to confirm the columns were dropped
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()

# Display the result
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
value_counts = application_df['APPLICATION_TYPE'].value_counts()

# Define a threshold for replacing with 'Other'
threshold = 1

# Identify types to replace with 'Other'
types_to_replace = value_counts[value_counts < threshold].index

# Replace identified types with 'Other'
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(types_to_replace, 'Other')

# Display the updated value counts to confirm changes
updated_value_counts = application_df['APPLICATION_TYPE'].value_counts()
print(updated_value_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
value_counts = application_df['APPLICATION_TYPE'].value_counts()

# Define a cutoff value
cutoff = 276

# Create a list of application types to be replaced
application_types_to_replace = value_counts[value_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
updated_value_counts = application_df['APPLICATION_TYPE'].value_counts()
print(updated_value_counts)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = application_df['CLASSIFICATION'].value_counts()

# Define a cutoff value
cutoff = 1

# Identify classifications to be replaced with 'Other'
classifications_to_replace = classification_value_counts[classification_value_counts < cutoff].index.tolist()

# Replace identified classifications with 'Other'
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other')

# Verify the replacement by checking the updated value counts
updated_classification_value_counts = application_df['CLASSIFICATION'].value_counts()

# Display the updated value counts
print(updated_classification_value_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts = application_df['CLASSIFICATION'].value_counts()

# Filter to get classifications with counts greater than 1
classification_counts_gt_1 = classification_value_counts[classification_value_counts > 1]

# Display the filtered value counts
print(classification_counts_gt_1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
value_counts = application_df['CLASSIFICATION'].value_counts()

# Define a cutoff value
cutoff = 1883

# Create a list of classifications to be replaced with "Other"
classifications_to_replace = value_counts[value_counts < cutoff].index.tolist()

# Replace identified classifications with "Other"
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other')

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
updated_classification_value_counts = application_df['CLASSIFICATION'].value_counts()

# Display the updated value counts to confirm changes
print(updated_classification_value_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [9]:
# Define the list of categorical columns to convert
categorical_columns = [
    'APPLICATION_TYPE',
    'AFFILIATION',
    'CLASSIFICATION',
    'USE_CASE',
    'ORGANIZATION',
    'STATUS',
    'INCOME_AMT',
    'SPECIAL_CONSIDERATIONS'
]

# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df, columns=categorical_columns)

# Check the first few rows of the transformed DataFrame
print(application_df_encoded.head())

   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0     5000              1                   False                  True   
1   108590              1                   False                 False   
2     5000              0                   False                 False   
3     6692              1                   False                 False   
4   142590              1                   False                 False   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                 False                False                False   
1                 False                 True                False   
2                 False                False                False   
3                 False                 True                False   
4                 False                 True                False   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
0                False                False                Fa

In [11]:
# Perform one-hot encoding
application_df_encoded = pd.get_dummies(application_df,
                                        columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
                                                 'USE_CASE', 'ORGANIZATION', 'STATUS',
                                                 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Training features shape: (27439, 44)
Testing features shape: (6860, 44)
Training target shape: (27439,)
Testing target shape: (6860,)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32

nn = tf.keras.models.Sequential()

# First hidden layer with dropout
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
nn.add(tf.keras.layers.Dropout(0.5))

# Second hidden layer with dropout
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)
nn.add(tf.keras.layers.Dropout(0.5))

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

# Print the model summary
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 128)                 │           5,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,129 (63.00 KB)

 Trainable params: 16,129 (63.00 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=["accuracy"])

In [28]:
# Define early stopping and learning rate scheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_schedule = LearningRateScheduler(lambda epoch: 1e-4 * 0.9 ** epoch)

# Train the model
history = nn.fit(
    X_train, y_train,
    epochs=200,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping, lr_schedule]
)

Epoch 1/200
343/343 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5497 - loss: 0.7015 - val_accuracy: 0.7013 - val_loss: 0.6421 - learning_rate: 1.0000e-04
Epoch 2/200
343/343 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6219 - loss: 0.6588 - val_accuracy: 0.7192 - val_loss: 0.6074 - learning_rate: 9.0000e-05
Epoch 3/200
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6595 - loss: 0.6397 - val_accuracy: 0.7228 - val_loss: 0.5894 - learning_rate: 8.1000e-05
Epoch 4/200
343/343 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.6743 - loss: 0.6360 - val_accuracy: 0.7276 - val_loss: 0.5797 - learning_rate: 7.2900e-05
Epoch 5/200
343/343 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6757 - loss: 0.6322 - val_accuracy: 0.7278 - val_loss: 0.5748 - learning_rate: 6.5610e-05
Epoch 6/200
343/343 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6931 - loss: 0.6141 - val_accuracy: 0.7307 - val_loss: 0.5713 - learning_rate: 5.9049e-05
Epoch 7/200
343/343 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - ac

In [29]:
# Evaluate the model using the test data
loss, accuracy = nn.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7246 - loss: 0.5701
Loss: 0.5717820525169373, Accuracy: 0.7250728607177734


In [30]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')